# Comprehensive Analysis of Datasets with Various Metrics, Norms, and Methods

In [ ]:
import sys
# sys.path.append('../')

import numpy as np
from AnalysisTools import Ana
from AnalysisTools import ComputeHelpersCPU
from AnalysisTools import ComputeHelpersGPU

# Initialize the analysis object
cachePath = '/home/diego/disks/ANACACHE'
comp = ComputeHelpersCPU(memory_location=cachePath, memory_verbosity=0, n_jobs=12)
compgpu = ComputeHelpersGPU()
analysis = Ana(showPlots=True, execution_mode=comp, cacheStoragePath=cachePath)


# cell lines in michrom and adam seperate / and  michrom vs adam


# Add datasets
# _2 is the dataset with higher error rate but looks visually better on the hic map

#IMR90
analysis.add_dataset(label="IMR90SIM",folder="/home/diego/disks/data/cndb/IMR90SIM/output")
analysis.add_dataset(label='IMR90OPT',folder='/home/diego/disks/data/IMR90OPT/lambda255to256')
# analysis.add_dataset(label="IMR90OPT_2",folder='/home/diego/disks/data/IMR90OPT/lambda56to57')

#GM12878
analysis.add_dataset(label="GM12878SIM",folder='/home/diego/disks/data/cndb/GM12878')
analysis.add_dataset(label="GM12878OPT",folder='/home/diego/disks/data/GM12878/lambda202to203')
# analysis.add_dataset(label="GM12878OPT_2",folder='/home/diego/disks/data/GM12878/lambda54to55')

#K562
analysis.add_dataset(label="K562SIM",folder='/home/diego/disks/data/cndb/K562')
analysis.add_dataset(label="K562OPT",folder='/home/diego/disks/data/K562/lambda270to271')
# analysis.add_dataset(label="K562OPT_2", folder='/home/diego/disks/data/K562/lambda65to66')

# #HELA
# analysis.add_dataset(label="HELASIM", folder='/home/diego/disks/data/cndb/HELA')
# analysis.add_dataset(label="HELAOPT", folder='/home/diego/disks/data/HELA/lambda293to294')
# analysis.add_dataset(label="HELAOPT_2", folder='/home/diego/disks/data/HELA/lambda69to70')

#STEM
analysis.add_dataset(label="STEMSIM", folder='/home/diego/disks/data/cndb/STEM')
analysis.add_dataset(label="STEMOPT", folder='/home/diego/disks/data/STEM/lambda300to301')
# analysis.add_dataset(label="STEMOPT_2", folder='/home/diego/disks/data/STEM/lambda73to74')

dataset_name = ["IMR90", "GM12878", "K562", "STEM"]

# Process Trajectories 
for name in dataset_name:
    if name == 'IMR90':
        analysis.process_trajectories(label=f'{name}SIM', cache_trajs=True, filename=f'traj_chr_{name}OPT_0.cndb', folder_pattern=['iteration_', [1, 20]])        
        analysis.process_trajectories(label=f'{name}OPT', cache_trajs=True, filename='traj_0.cndb', folder_pattern=['iteration_', [1,20]])
        # analysis.process_trajectories(label=f'{name}OPT_2', cache_trajs=True, filename='traj_0.cndb', folder_pattern=['iteration_', [1,20]])
        continue
    analysis.process_trajectories(label=f'{name}OPT', cache_trajs=True, filename='traj_0.cndb', folder_pattern=['iteration_', [1,20]])
    # analysis.process_trajectories(label=f'{name}OPT_2', cache_trajs=True, filename='traj_0.cndb', folder_pattern=['iteration_', [1,20]])
    analysis.process_trajectories(label=f'{name}SIM', cache_trajs=True, filename=f'traj_chr_{name}_0.cndb', folder_pattern=['iteration_', [1, 20]])


In [ ]:
# norms = ['ice', 'kr', 'log_transform', 'vc']
# metrics = ['euclidean', 'pearsons', 'spearman', 'contact', 'log2_contact']


norms = ['log_transform', 'vc']
metrics = ['log2_contact', 'contact', 'euclidean']
methods = ['single', 'weighted']

# seq = [["IMR90OPT", "IMR90SIM"], ["K562OPT", "K562SIM"],
#        ["HELAOPT", "HELASIM"], ["STEMOPT",  "STEMSIM"], ["GM12878OPT", "GM12878SIM"] ]

# seq = [["IMR90OPT", "GM12878OPT"], ["IMR90OPT", "GM12878OPT", "STEMOPT"], ["IMR90OPT", "GM12878OPT", "STEMOPT", "K562OPT"]]
seq = [["IMR90SIM", "GM12878SIM"], ["IMR90SIM", "GM12878SIM", "STEMSIM"], ["IMR90SIM", "GM12878SIM", "STEMSIM", "K562SIM"]]


# labels = [["IMR90 SIM", "IMR90 OPT"], ["K562 SIM", "K562 OPT"], ["HELA SIM", "HELA OPT"], ["STEM SIM", "STEM OPT"], ["GM12878 SIM", "GM12878 OPT"]]


## Generate and Cache Distance Matrices

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            for ele in seq:
                if len(ele) == 2:
                    analysis.calc_XZ(ele[0], ele[1], metric=metric, norm=norm, method=method)

                elif len(ele) == 3:
                    analysis.calc_XZ(ele[0], ele[1], ele[2], metric=metric, norm=norm, method=method)

                elif len(ele) == 4:
                    analysis.calc_XZ(ele[0], ele[1], ele[2], ele[3], metric=metric, norm=norm, method=method)



## Dimensionality Reduction Techniques

### PCA Analysis

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            for ele in seq:
                if len(ele) == 2:
                    analysis.pca(ele[0], ele[1], metric=metric, n_components=1, norm=norm, method=method, n_clusters=2, labels=seq[0])

                elif len(ele) == 3:
                    analysis.pca(ele[0], ele[1], ele[2], metric=metric, n_components=1, norm=norm, method=method, n_clusters=3, labels=seq[1])

                elif len(ele) == 4:
                    analysis.pca(ele[0], ele[1], ele[2], ele[3], metric=metric, n_components=1, norm=norm, method=method, n_clusters=4, labels=seq[2])                    

 


## Expiremental Scaler with best pca norm x method x metric dataset

In [ ]:
#_weighted_euclidean_log_transform.png
metrics = ['euclidean', 'pearsons', 'spearman', 'contact', 'log2_contact']
norms = ['ice', 'kr', 'log_transform', 'vc']
from AnalysisTools.Plot_Helper import PlotHelper

for norm in norms:
    for metric in metrics:
        X, Z = analysis.calc_XZ("IMR90SIM", "IMR90OPT", "IMR90OPT57", method="weighted", metric=metric, norm=norm, overrideCache=True, expiremental=True)
        n_components = 1
        pca, exp, com = comp.run_reduction('pca', X, 1)

        plot_params = {
            'outputFileName': f"test00_{norm}_{metric}",
            'cmap': 'viridis',
            'title': f'PCA of test',
            'x_label': 'PC1',
            'y_label': 'PC2' if n_components > 1 else 'Principal Component 1',
            'z_label': 'PC3' if n_components > 2 else None,
            'n_components': n_components,
            'n_clusters': 2,
            'method': "weighted",
            'metric': "euclidean",
            'norm': "log_transform",
            'n_components_95': 1,
            'size': 50,
            'alpha': 0.7,
        }

        if n_components > 1:
            plot_params['y_label'] = f'PC2 ({exp[1]:.2%} variance)'
        else:
            plot_params['y_label'] = 'Samples'


        pl = PlotHelper()
        pl.plot(plot_type="pcaplot", data=(pca, exp, com), plot_params=plot_params)


### UMAP Analysis

### t-SNE Analysis

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.tsne("IMR90OPT", "IMR90OPT57", "IMR90SIM", metric=metric, n_clusters=2, norm=norm, method=method, n_components=1)

### MDS Analysis

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.mds("IMR90OPT", "IMR90OPT57", "IMR90SIM", metric=metric, n_components=1, norm=norm, method=method)

### SVD Analysis

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.svd("IMR90OPT", "IMR90OPT57", "IMR90SIM", metric=metric, n_components=1, norm=norm, method=method, n_clusters=2)

## Clustering Techniques

### K-means Clustering

In [ ]:
for norm in norms:
    for metric in metrics:
            analysis.kmeans_clustering("IMR90OPT", "IMR90OPT57", "IMR90SIM", n_clusters=2, metric=metric, norm=norm)

### DBSCAN Clustering

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.dbscan_clustering("IMR90OPT", "IMR90OPT56", "IMR90SIM", eps=0.5, min_samples=5, metric=metric, norm=norm, method=method)

### Hierarchical Clustering

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.hierarchical_clustering("IMR90OPT", "IMR90OPT56", "IMR90SIM", n_clusters=5, metric=metric, norm=norm, method=method)

In [ ]:
for norm in norms:
    for metric in metrics:
            analysis.spectral_clustering("IMR90OPT", "IMR90OPT57", "IMR90SIM", n_clusters=2, n_components=2, metric=metric, norm=norm)

### Spectral Clustering

In [ ]:
for norm in norms:
    for metric in metrics:
            print(analysis.spectral_clustering("IMR90OPT", "IMR90OPT57", "IMR90SIM", n_clusters=-1, n_components=1, metric=metric, norm=norm))

### OPTICS Clustering

In [ ]:
for norm in norms:
    for metric in metrics:
        for method in methods:
            analysis.optics_clustering("IMR90OPT", "IMR90OPT56", "IMR90SIM", min_samples=5, xi=0.05, min_cluster_size=0.05, metric=metric, norm=norm, method=method)